In [1]:
from parafoil import CamberThicknessAirfoil
from parafoil.passages import TurboRowPassage, TurboStationPassage


mean_rotor_airfoil = CamberThicknessAirfoil(
    inlet_angle=-60,
    outlet_angle=-51,
    upper_thick_prop=[0.015, 0.05, 0.05, 0.015],
    lower_thick_prop=[0.015, 0.05, 0.05, 0.015],
    leading_prop=0.5,
    trailing_prop=0.5,
    chord_length=0.018,
    angle_units="deg"
)
mean_rotor_pasage = TurboStationPassage(
    airfoil=mean_rotor_airfoil,
    spacing_to_chord=1.0,
    leading_edge_gap_to_chord=4,
    trailing_edge_gap_to_chord=4,
    type="camber",
)
hub_rotor_airfoil = mean_rotor_airfoil.clone(inlet_angle=-50, outlet_angle=-25)
tip_rotor_airfoil = mean_rotor_airfoil.clone(inlet_angle=-67, outlet_angle=-63)

hub_rotor_passage = mean_rotor_pasage.clone(airfoil=hub_rotor_airfoil)
tip_rotor_passage = mean_rotor_pasage.clone(airfoil=tip_rotor_airfoil)
blade = TurboRowPassage([hub_rotor_passage, mean_rotor_pasage, mean_rotor_pasage], radii=[0.1131, 0.1697, 0.2262])

turbo_profile = blade.get_profile()

import cadquery as cq
cq.exporters.export(turbo_profile, "rotor67.step")

from jupyter_cadquery import show
show(turbo_profile)


objc[75146]: Class vtkCocoaTimer is implemented in both /opt/miniconda3/envs/brad/lib/python3.11/site-packages/cadquery_ocp_arm.dylibs/libvtkRenderingUI-9.2.9.2.6.dylib (0x114bb0410) and /opt/miniconda3/envs/brad/lib/python3.11/site-packages/vtkmodules/.dylibs/libvtkRenderingUI-9.2.dylib (0x1414f03e8). One of the two will be used. Which one is undefined.
objc[75146]: Class vtkCocoaFullScreenWindow is implemented in both /opt/miniconda3/envs/brad/lib/python3.11/site-packages/cadquery_ocp_arm.dylibs/libvtkRenderingOpenGL2-9.2.9.2.6.dylib (0x1179f5f88) and /opt/miniconda3/envs/brad/lib/python3.11/site-packages/vtkmodules/.dylibs/libvtkRenderingOpenGL2-9.2.dylib (0x145cd2020). One of the two will be used. Which one is undefined.
objc[75146]: Class vtkCocoaServer is implemented in both /opt/miniconda3/envs/brad/lib/python3.11/site-packages/cadquery_ocp_arm.dylibs/libvtkRenderingOpenGL2-9.2.9.2.6.dylib (0x1179f5fb0) and /opt/miniconda3/envs/brad/lib/python3.11/site-packages/vtkmodules/.dylib

Overwriting auto display for cadquery Workplane and Shape


CadViewerWidget(anchor=None, cad_width=800, glass=False, height=600, pinning=False, theme='light', title=None,…

In [4]:
from typing import List, Optional, Sequence, Union
from meshql import GeometryQL, Selection, Split
import cadquery as cq
from jupyter_cadquery import show_object
from parafoil.boundary_conditions import ConstantHeatfluxBoundaryCondition, InletTotalBoundaryCondition, MonitoringBoundaryCondition, OutletStaticPressureBoundaryCondition, PeriodicBoundaryCondition, PlottingBoundaryCondition, ShroudBoundaryCondition, TurbomachineryBoundaryCondition

interior_right_side = Selection(type="interior", filter=lambda edge: edge.Center().x>0)
interior_left_side = Selection(type="interior", filter=lambda edge: edge.Center().x<0)

profile_bbox = turbo_profile.val().BoundingBox()

# print(mean_rotor_pasage.leading_edge_gap)


class DirectionSelector(cq.selectors.Selector):
    def __init__(self, parent: cq.Workplane, selector_str: str) -> None:
        super().__init__()
        self.parent= parent
        self.bbox = parent.val().BoundingBox()
        self.bbox_center = parent.val().Center()
        self.selector_str = selector_str
        self.bbox_profile = cq.Workplane("XY", origin=self.bbox_center).box(self.bbox.xlen*2, self.bbox.ylen*2, self.bbox.zlen*2)

    def filter(self, objectList: Sequence[cq.Face]) -> List[cq.Face]:
        filtered_faces = []

        selector_face = self.bbox_profile.faces(self.selector_str).val()

        for face in objectList:
            faces = selector_face.facesIntersectedByLine(face.Center(), face.normalAt(), direction="AlongAxis")
            if len(faces) and faces[0] == selector_face:
                filtered_faces.append(face)

        return filtered_faces

XYZRangeType = Optional[tuple[Optional[float], Optional[float]]]

class XYZRangeSelector(cq.selectors.Selector):
    def __init__(self, xyz_range: tuple[XYZRangeType, XYZRangeType, XYZRangeType]) -> None:
        super().__init__()
        self.xyz_range = xyz_range

    def is_within_range(self, coord: float, coord_range: XYZRangeType) -> bool:
        """Utility method to check if a coordinate is within the specified range."""
        if coord_range is None:
            return True
        min_val, max_val = coord_range
        return (min_val is None or coord >= min_val) and (max_val is None or coord <= max_val)

    def filter(self, object_list: Sequence[cq.Shape]) -> List[cq.Face]:
        filtered_faces = []
        for shape in object_list:
            center = shape.Center()
            if all([
                self.is_within_range(center.x, self.xyz_range[0]),
                self.is_within_range(center.y, self.xyz_range[1]),
                self.is_within_range(center.z, self.xyz_range[2])
            ]):
                filtered_faces.append(shape)
                
        return filtered_faces



leading_x = mean_rotor_pasage.leading_edge_gap*0.2
trailing_x = -mean_rotor_pasage.trailing_edge_gap*0.2

# TODO: fix the meshql cache stuff
with GeometryQL.gmsh() as geo:
    new_geo = (
        geo
        .load(
            turbo_profile,
            on_preprocess= lambda ql: (
                Split(ql)
                .from_plane(base_pnt=(0.072*0.2, 0), angle=(90, 0, 0))
                .from_plane(base_pnt=(-0.072*0.2, 0), angle=(90, 0, 0))
                .group(
                    lambda ql: (
                        Split(ql)
                        .from_ratios(
                            start_select=Selection(">Z", type="interior"), 
                            end_select=Selection("<Z", type="interior"),
                            ratios=[0.45, 0.95],
                            dir="away",
                            snap=True,
                        )
                        .from_normals(
                            Selection(type="interior", filter=lambda edge: edge.Center().x<0), 
                            axis=[(-1, 0, 0), (0, 1, 0)]
                        )
                        .from_normals(
                            Selection(type="interior", filter=lambda edge: edge.Center().x>0), 
                            axis=[(1, 0, 0), (0.5, -1, 0)]
                        )
                    )
                )
            ),
        )
        .faces(">X")
        .addBoundaryCondition(
           lambda i, face:  InletTotalBoundaryCondition(
                label="inlet",
                total_temperature=300,
                total_pressure=101325,
                flow_direction=(-face.normalAt()).toTuple(),
            )
        )
        .end()

        .faces("<X")
        .addBoundaryCondition(
            OutletStaticPressureBoundaryCondition(
                label="outlet",
                static_pressure=101325,
            )
        )
        .end()

        # add periodic boundary conditions
        .faces(DirectionSelector(turbo_profile, ">Y"), type="exterior")
        .addPhysicalGroup("periodic1")
        .end()

        .faces(DirectionSelector(turbo_profile, "<Y"), type="exterior")
        .addPhysicalGroup("periodic2")
        .end()

        .addBoundaryCondition(
            PeriodicBoundaryCondition(
                periodic_label="periodic1",
                donor_label="periodic2",
            )
        )



        .faces(">Z")
        .addBoundaryCondition(
            ConstantHeatfluxBoundaryCondition(
                label="shroud"
            )
        )
        .end()


        .faces(type="interior")
        .addBoundaryCondition(
            ConstantHeatfluxBoundaryCondition(
                label="blade"
            )
        )
        .end()


        # add hub boundary conditions
        .faces("<Z")
        
        .faces(XYZRangeSelector(((trailing_x, leading_x), None, None)))
        .addBoundaryCondition(ConstantHeatfluxBoundaryCondition(label="hub"))
        .end()

        .faces(XYZRangeSelector(((leading_x, None), None, None)))
        .addBoundaryCondition(ConstantHeatfluxBoundaryCondition(label="hub_upstream"))
        .end()

        .faces(XYZRangeSelector(((None, trailing_x), None, None)))
        .addBoundaryCondition(ConstantHeatfluxBoundaryCondition(label="hub_downstream"))
        .end()

        .end()

        .addBoundaryCondition(ShroudBoundaryCondition(shroud_labels=["shroud", "hub_upstream", "hub_downstream"]))
        # .addBoundaryCondition(TurbomachineryBoundaryCondition(inflow_label="inlet", outlet_label="outlet"))
        .addBoundaryCondition(MonitoringBoundaryCondition(monitoring_labels=["inlet"]))
        .addBoundaryCondition(PlottingBoundaryCondition(plotting_labels=["inlet","outlet","hub","blade"]))

        .setTransfiniteAuto(max_nodes=200)
        .generate(3)
        # .show("gmsh")
        .show("mesh", only_markers=True)
    )



Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (BSpline)
Info    : [ 10%] Meshing curve 2 (BSpline)
Info    : [ 10%] Meshing curve 3 (BSpline)
Info    : [ 10%] Meshing curve 4 (BSpline)
Info    : [ 10%] Meshing curve 5 (BSpline)
Info    : [ 10%] Meshing curve 6 (BSpline)
Info    : [ 10%] Meshing curve 7 (BSpline)
Info    : [ 10%] Meshing curve 8 (Line)
Info    : [ 20%] Meshing curve 9 (Line)
Info    : [ 20%] Meshing curve 10 (BSpline)
Info    : [ 20%] Meshing curve 11 (Line)
Info    : [ 20%] Meshing curve 12 (Line)
Info    : [ 20%] Meshing curve 13 (BSpline)
Info    : [ 20%] Meshing curve 14 (BSpline)
Info    : [ 20%] Meshing curve 15 (BSpline)
Info    : [ 30%] Meshing curve 16 (BSpline)
Info    : [ 30%] Meshing curve 17 (BSpline)
Info    : [ 30%] Meshing curve 18 (BSpline)
Info    : [ 30%] Meshing curve 19 (Line)
Info    : [ 30%] Meshing curve 20 (Line)
Info    : [ 30%] Meshing curve 21 (BSpline)
Info    : [ 30%] Meshing curve 22 (BSpline)
Info    : [ 30%] Meshing curve 23 (

HTML(value='Coords: ()')

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, far=100000.0, near=0.001, position=(0.0, 0.0, 1.0…

,outlet
,periodic1
,hub_downstream
,shroud
,hub
,periodic2
,blade
,hub_upstream
,inlet
,Zone 0


In [4]:
from meshql import GeometryQL, Selection, Split

from meshql.utils.cq import CQUtils
CQUtils.max_dim_multiplier = 10
with GeometryQL.gmsh() as geo:
    out = (
        geo
        .load(
            "/Users/afshawnlotfi/Documents/parafoil/rotor67.step",
            on_preprocess= lambda ql: (
                Split(ql)
                .from_plane(base_pnt=(0.072*0.2, 0), angle=(90, 0, 0))
                .from_plane(base_pnt=(-0.072*0.2, 0), angle=(90, 0, 0))
                .group(
                    lambda ql: (
                        Split(ql)
                        .from_ratios(
                            start_select=Selection(">Z", type="interior"), 
                            end_select=Selection("<Z", type="interior"),
                            ratios=[0.45, 0.95],
                            dir="away",
                            snap=True,
                        )
                        .from_normals(
                            Selection(type="interior", filter=lambda edge: edge.Center().x<0), 
                            axis=[(-1, 0, 0), (0, 1, 0)]
                        )
                        .from_normals(
                            Selection(type="interior", filter=lambda edge: edge.Center().x>0), 
                            axis=[(1, 0, 0), (0.5, -1, 0)]
                        )
                    )
                )
            ),
        )

        .setTransfiniteAuto(max_nodes=200)
        
        .faces(type="interior")
        .show()
        .addBoundaryLayer(0.001)
        .end()

        .generate(3)
        .show("mesh", only_markers=True)
        # .show("gmsh")
    )
# test()
# cProfile.run("test()", sort="cumtime")

100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.18s


CadViewerWidget(anchor=None, cad_width=800, glass=False, height=600, pinning=False, theme='light', title=None,…

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (BSpline)
Info    : [ 10%] Meshing curve 2 (BSpline)
Info    : [ 10%] Meshing curve 3 (BSpline)
Info    : [ 10%] Meshing curve 4 (BSpline)
Info    : [ 10%] Meshing curve 5 (BSpline)
Info    : [ 10%] Meshing curve 6 (BSpline)
Info    : [ 10%] Meshing curve 7 (BSpline)
Info    : [ 10%] Meshing curve 8 (Line)
Info    : [ 20%] Meshing curve 9 (Line)
Info    : [ 20%] Meshing curve 10 (BSpline)
Info    : [ 20%] Meshing curve 11 (Line)
Info    : [ 20%] Meshing curve 12 (Line)
Info    : [ 20%] Meshing curve 13 (BSpline)
Info    : [ 20%] Meshing curve 14 (BSpline)
Info    : [ 20%] Meshing curve 15 (BSpline)
Info    : [ 30%] Meshing curve 16 (BSpline)
Info    : [ 30%] Meshing curve 17 (BSpline)
Info    : [ 30%] Meshing curve 18 (BSpline)
Info    : [ 30%] Meshing curve 19 (Line)
Info    : [ 30%] Meshing curve 20 (Line)
Info    : [ 30%] Meshing curve 21 (BSpline)
Info    : [ 30%] Meshing curve 22 (BSpline)
Info    : [ 30%] Meshing curve 23 (

HTML(value='Coords: ()')

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, far=100000.0, near=0.001, position=(0.0, 0.0, 1.0…

,Zone 0
